# Tutorial: Pandas for software engineering research

- **Auhtor**: Felipe Ortega.
- **Date**: June 7, 2017.
- **Venue**: SATToSE 2017 Workshop.
- **Location**: URJC. Madrid.
- **Part**: 1 of 2.

## 1. Introduction

Pandas is one of the most popular libraries for **Python scientific programming**. In this tutorial, we explore the use of `Pandas` to load, prepare and analyse structured datasets, using the **`DataFrame` data structure** that has become a ubiquitous tool in data science.

To follow this tutorial, you should have [**Anaconda Python**](https://www.continuum.io/downloads) installed in your system (64-bit version, probably for Python 3.6). You should also create a new virtual environment with `conda`, either [using the command line](https://conda.io/docs/using/envs.html#create-an-environment) or using [Anaconda Navigator](https://docs.continuum.io/anaconda/navigator/). In this way, you will have a separate virtual space to play with these examples. If you already have a favourite IDE for Python programming, you can [follow the instructions to use Anaconda in your IDE](https://docs.continuum.io/anaconda/ide_integration).

## 2. Loading your data

The SATToSE 2017 workshop presents a dataset of 250K recent Scratch projects from 100K different authors scraped from the Scratch project repository. You can find this dataset [on GitHub](https://github.com/TUDelftScratchLab/ScratchDataset).

Data from Scratch projects are available in different formats, stored in the GitHub repository presented above:
* Raw data in [CSV files](https://github.com/TUDelftScratchLab/ScratchDataset/tree/master/Dataset/CSV%20files).
* A compressed (gzip) [MySQL dump file](https://github.com/TUDelftScratchLab/ScratchDataset/tree/master/Dataset/mySQL).
* A compressed (zip) [SQL server backup file](https://github.com/TUDelftScratchLab/ScratchDataset/tree/master/Dataset/SQL%20Server).

In this tutorial, we use the second option and upload all data in a **local MySQL database**, named `Scratch_data`.

Once you have created this local database and uploaded the Scratch dataset into it, you can continue following this tutorial.

### 2.1 Python DB API

In Python, there are several packages that we can use to communicate with popular RDBMS products, such as MySQL/MariaDB or PostgreSQL. An advantage is that many of these packages stick to a standard high-level interface to communicate with any database, defined in [PEP 249 -- Python Database API Specification v2.0](https://www.python.org/dev/peps/pep-0249/). Hence, we can be confident to find the same methods in any of these packages, disregarding of which particular database we need to interact with.

There are several Python packages that provide connectivity between Python and MySQL. In [this article from OpenStack](https://wiki.openstack.org/wiki/PyMySQL_evaluation) you can find a nice comparison among some of these alternatives. In essence, all Python connectors for MySQL are divided into native Python connectors (written entirely in pure Python) and wrappers around starndard MySQL connector library written in C. However, **we must be careful** if we aim at **using Python 3**, since **some of the most popular packages are not compatible** with this Python version. You can also consider some of the ORM frameworks in Python to use these libraries, such as [SQLAlchemy](https://www.sqlalchemy.org/).

For the sake of simplicity, in this tutorial we use [`PyMySQL`](https://github.com/PyMySQL/PyMySQL), a pure Python MySQL client. Please, bear in mind that this is the most straightforward option for easy installation and maintenance (no external requirements, such as low-level C library package) but, regarding performance, it is probably one of the slowest alternatives available. It is also fully compliant with Python 3${^1}$.

${^1}$: *If performance is a major concern for you, then you should probably use one of the options wrapped on top of the low-level C API for MySQL, always watching for compatibility with Python 3 (if needed)*. One example is [mysqlclient-python](https://github.com/PyMySQL/mysqlclient-python).

To install PyMySQL, type the following in a command-line where you have activated your virtual environment in Anaconda (in the example below, the virtual environment is `datascience`). In Windows, you can also activate your virtual environment using Anaconda Navigator:

```bash
$ source ~/anaconda3/bin/activate datascience
(datascience)$ pip install PyMySQL
```

### 2.2 DB connection

Now, we procceed to connect to our local database and get some data using `PyMySQL`:

In [16]:
import pymysql.cursors

# Get new connection with local MySQL DB
connection = pymysql.connect(host='localhost',
                             user='jfelipe',
                             password='phoenix',
                             db='Scratch_data',
                             charset='utf8mb4',
                             cursorclass=pymysql.cursors.DictCursor)

try:
    with connection.cursor() as cursor:
        # Read some data
        sql = """SELECT `p_ID`, `total-views`, `total-remixes`
                 FROM `project` LIMIT 10"""
        cursor.execute(sql)
        result = cursor.fetchall()
        print(result)
finally:
    connection.close()

[{'p_ID': 10519701, 'total-views': 4.0, 'total-remixes': 0.0}, {'p_ID': 10541430, 'total-views': 2.0, 'total-remixes': 0.0}, {'p_ID': 10942522, 'total-views': 3.0, 'total-remixes': 0.0}, {'p_ID': 11823040, 'total-views': 25.0, 'total-remixes': 0.0}, {'p_ID': 11828235, 'total-views': 1.0, 'total-remixes': 0.0}, {'p_ID': 12030146, 'total-views': 15.0, 'total-remixes': 0.0}, {'p_ID': 12093755, 'total-views': 1.0, 'total-remixes': 0.0}, {'p_ID': 12237615, 'total-views': 4.0, 'total-remixes': 0.0}, {'p_ID': 12256407, 'total-views': 2.0, 'total-remixes': 0.0}, {'p_ID': 12441584, 'total-views': 3.0, 'total-remixes': 0.0}]


The `result` object stores all values returned by the query, whereas the `cursor.description` attribute of the `cursor` object contains metadata about the columns returned by the query.

In [12]:
# Get cursor description
# See: https://www.python.org/dev/peps/pep-0249/#cursor-attributes
cursor.description

(('p_ID', 8, None, 20, 20, 0, False),
 ('total-views', 5, None, 22, 22, 31, True),
 ('total-remixes', 5, None, 22, 22, 31, True))

Now we can start using the specific modules for scientific programming in Python to analyse our data.

## 3. Scientific programming with Python

There exist many different modules supporting the complete technology stack for Python scientific programming. Figure 1 below depicts the diagram showing the different components in the stack.

<img src="scientific-python-28-638.jpg" width=600>

<center>**Figure 1**: Technology stack in Python scientific programming (source: [Jake VanderPlas](https://staff.washington.edu/jakevdp/), [“The State of the Stack”](https://speakerdeck.com/jakevdp/the-state-of-the-stack-scipy-2015-keynote), keynote in *SciPy 2015*).</center>

On top of the core Python interpreter we find a **first layer** of projects providing support for all **basic features**:
* **IPython**: Enhnaced interactive interpreter for Python programming, with special emphasis on scientific applications.
* **Jupyter**: Web application that enables the creation of interactive notebooks (like this document), implementing (and extending) the principles of [*literate programming*](https://en.wikipedia.org/wiki/Literate_programming). Jupyter notebooks are interactive, and they can be edited and extended to your own needs, facilitating knowledge sharing and management. Currently, Jupyter features support for 40 programming languages, including (besides Python) R, Julia or Scala.
* **NumPy**: This is the baseline module for numerical calculations in Python scientific programming. It provides support for working with N-dimensional array objects, fast computations through integration of C/C++ and Fortran code or specialised functions and methods (e.g. Fourier transforms, random numbers or linear algebra).

The **second layer** of the Python scientific stack includes several modules that build on top of the first layer to solve more complex but still **fundamental features**:

* **SciPy**: This is a library comprising several modules implementing more [complex and specialized scientific operations](https://docs.scipy.org/doc/scipy/reference/), on top of NumPy. For example, we can find support for integration, optimization, signal processing or graph routines.
* **Matplotlib**: This is the baseline data visualization service for Python scientific programming.
* **Pandas**: A library implementing fundamental data structures and operations to deal with structured datasets (similar to database tables). This is the main component that we use in this tutorial.

Beyond these two first layers, we can find **high-level projects** that tackle more complex tasks. Among these projects, we can highlight:

* **StatsModels**: Implementation of statistical models and algorithms.
* **Scikit-learn**: Data mining and machine learning algorithms and routines. It also includes standard procedures to define and implement [data pipelines](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).
* **PyTables**: Support for hierarchical and very large datasets (structured data).

In this tutorial, we will focus on `Pandas` to work with regular data tables. Besides, we will also introduce `Seaborn`, an additional project to make it easier the creation of advanced data visualization graphs from `Pandas` objects. Finally, we will also introduce some cutting-edge projects recently created to undertake the increasingly popular need of processing large datasets on distributed computer infrastructures.

## 4. Introducing `Pandas`

`NumPy` and `SciPy` provide support for low-level programming operations and routines. The [**Pandas**](http://pandas.pydata.org/pandas-docs/stable/) library sits on top of them to provide all principal operations to deal with structured datasets, including data loading, data preparation and cleansing. Pandas also provides familiar data structures and operations for data scientists, such as **Series** for **indexed arrays of values** or the **DataFrame** class for data tables.

The name of the Pandas library is derived, in part from **panel data** (pan(el)-da(ta)-s). This term is usually found in econometrics and social sciences to name data tables organized in a way similar to a table in a relational database:

* One case for each row.
* One variable for each column, defined as a fixed data type.

The main advantages of using `Pandas` for data cleaning and analytics are working with standard data structures, methods and procedures to work with **structured datasets**, including data from relational databases or labeled data.

The main use cases of `Pandas` are:

* Working with data tables with columns of different types (but always using the same type for all values in a single column). This is similar to the `data.frame` objects in the R programming language, that in fact inspired the creation of the `DataFrame` class in `Pandas`.
* Working with time series data, both regular or irregular, and not necessarily ordered.
* Working with matrices, with rows and columns that can be labeled (again, pretty much like in R).
* In general, it is adequate for working with most of the datasets that can be found in data science studies.

Among its **main features** we can mention:

* Handling missing data.
* Mutable size of objects. We can add or delete columns or rows in N-dimensional collections of objects and primitive types.
* Explicit data alignment, according to a series of ordered labels (such as integers, strings or timestamps), or totally automatic alignment.
* Applying *split-and-combine* operations, similar to the working philosophy of the `dplyr` and `tidyr` libraries in the R programming language.
* Powerful and efficient ways for indexing and creating subsets, based on labels or categories.
* Joining and integrating datasets (operations like *join* or *union*).
* Transforming data tables between *long* and *wide* formats (which is called *reshaping* and *pivoting* in `Pandas` terminology).
* Hierarchical labeling of dimensional axes.
* Wide variety of I/O functions, facilitating the reading/writing from/to data files in different formats, such as plain text files (CSV/TSV), MS Excel, databases or HDF5.
* Support for time series operations, like generating data at regular intervals, sampling frequency conversion, descriptive statistics and linear regression on sliding windows, data shifting and lagging, etc.

The `Pandas` library documentation presents a succint [10-minute introduction to `Pandas`](http://pandas.pydata.org/pandas-docs/stable/10min.html), with interesting examples to illustrate some of the basic data structures and methods.

To start working with `Pandas`, we only need to import the `NumPy` and `Pandas` libraries, following the standard convention in Python scientific programming:

In [23]:
import numpy as np
import pandas as pd

In [24]:
# Check version of both libraries
print("NumPy version: ", np.__version__)
print("Pandas version: ", pd.__version__)

NumPy version:  1.12.1
Pandas version:  0.20.1


## 5. Series objects

The main data structures implemented in `Pandas` are the `Series` class and the `DataFrame` class. In this section, we start describing `Series` objects.

### 5.1 Creating `Series` objects

`Series` objects contain a **data vector** (similar to a NumPy array), but they also have **identification labels for each element in the vector**, instead of using a basic integer index to order data. This is an important advantage, since we can reorder values in a `Series` object efficiently (using labels), or we can find values within the series (for a unique label). Furthermore, this data structure is ideal for manage **time series data** (series of values along with the timestamp for each value), as we will see in another section below.

[`Series` objects](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#series) can be created from different types of input data, described by the `data` argument, such as Python dictionaries, `ndarray` NumPy objects or scalar values.

```python
my_series = pd.Series(data, index=my_index)
```

Besides, we must provide an `index` argument, with a vector of labels to identify each of the values within the series. This spawns different use cases:

In [33]:
# If data is an ndarray, then the index vector musth be of the same length
series_1 = pd.Series(np.random.randn(5), index=['a', 'b', 'c', 'd', 'e'])
series_1

a   -0.583969
b   -0.074399
c   -0.455512
d    0.633556
e   -0.861814
dtype: float64

In [34]:
# The index attribute stores the vector of labels for the series of values
series_1.index

Index(['a', 'b', 'c', 'd', 'e'], dtype='object')

In [35]:
# The vector of values is stored in the values attribute
series_1.values

array([-0.58396939, -0.07439888, -0.45551169,  0.63355603, -0.86181351])

In [36]:
# If we do not provide an index vector of labels, then a numerical index is created automatically
series_2 = pd.Series(np.random.rand(5))
series_2

0    0.826285
1    0.307435
2    0.855067
3    0.866171
4    0.557727
dtype: float64

In [30]:
# If data is in a Python dictionary, then if we pass a vector of indexes
# then indexing labels will be used to take the elements in the dictionary
# according to the key of each element, in the same order in which
# the labels appear in the vector of indexes

# If no index vector is provided, labels are created from the ordered
# keys of the dictionary
d = {'a': 1., 'b': 2., 'c': 3.0, 'd': 4. }
pd.Series(d)

a    1.0
b    2.0
c    3.0
d    4.0
dtype: float64

In [31]:
# If one of the keys in the vector of indexes is not found as a key
# in the dictionary, a NaN value is automatically inserted in that position
pd.Series(d, index = ['e', 'd', 'c', 'b', 'a'])

e    NaN
d    4.0
c    3.0
b    2.0
a    1.0
dtype: float64

In [37]:
# We can also assign a name to the Series object
# This is useful when we create multi-dimensional data
# structures comprising several Series objects
series_2.name = "Series 2"
series_2.index.name = "Ordinal"
series_2

Ordinal
0    0.826285
1    0.307435
2    0.855067
3    0.866171
4    0.557727
Name: Series 2, dtype: float64

### 5.2 Indexing objects in `Pandas`

Any data structure in `Pandas` can be indexed in 3 different ways:

1. Indexing by label: https://pandas.pydata.org/pandas-docs/stable/indexing.html#selection-by-label.
2. Indexing by position: https://pandas.pydata.org/pandas-docs/stable/indexing.html#selection-by-position.
3. Indexing by *callable*: https://pandas.pydata.org/pandas-docs/stable/indexing.html#selection-by-callable.

Hence, we can use any of these options in either `Series` or `DataFrame` objects. An important remark is that we cannot mix different types of selection together in the same indexing expression.

### 5.3 Indexing `Series` objects

Indexing operations in `Series` objects are similar to the equivalent options in other languages: using numerical indexes, boolean expressions, etc. In Python, you can also use *slicing* for numerical or label indexes, provided that there is a strict ordering among their values.

Additionally, since indexing labels are stored alongside the vector of values in a `Series` object, we can use such labels like in a Python dictionary, to access individual elements (or a group of them). This option has de additional advantage that is very fast (low computational complexity) and the resulting code is quite clear and readable, since we are using meaningful identifiers to access values in the object.

In [42]:
# Indexing with numerical positions, individually or with slicing
series_1[0]

-0.58396939323693919

In [43]:
print(series_1[2:5])

c   -0.455512
d    0.633556
e   -0.861814
dtype: float64


In [44]:
print(series_1)

a   -0.583969
b   -0.074399
c   -0.455512
d    0.633556
e   -0.861814
dtype: float64


In [45]:
# Indexing with labels
series_1[['a', 'b']]

a   -0.583969
b   -0.074399
dtype: float64

In [47]:
# Indexing with boolean expressions
series_1[series_1 < -0.5]

a   -0.583969
e   -0.861814
dtype: float64

In [49]:
# Using boolean expression and callable (method median)
series_2[series_2 > series_2.median()]

Ordinal
2    0.855067
3    0.866171
Name: Series 2, dtype: float64

If we pay attention to results in the examples above, we can check that we not only get a subset of values, but also the corresponding subset of indexes corresponding to those values. That is, the *slicing* indexing is applied to both the vector of indexes and the vector of values inside the `Series` object. In this way, we can index without losing any of the special properties of this object.

### 5.4 Operations with `Series` objects

It is possible to perform arithmetic operations or functions over the values stored in a `Series` object, like we do in other objects in scientific Python programming such as `NumPy` arrays.

In [50]:
np.log10(series_2)

Ordinal
0   -0.082870
1   -0.512247
2   -0.068000
3   -0.062396
4   -0.253578
Name: Series 2, dtype: float64

In [51]:
# Add up element by element, using the index labels
# to match cases in each series
series_1 + series_1

a   -1.167939
b   -0.148798
c   -0.911023
d    1.267112
e   -1.723627
dtype: float64

## 6 `DataFrame` objects

The `Pandas` [`DataFrame` class](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) defines aa data structure to store and process datasets along the same lines of `data.frame` objects in the R programming language. It is a table of values organized such that:

* Each row corresponds to an individual case or element in the study.
* Each column is a variable in the study. All values in the column must be of the same type.

Like in a relational database or a CSV file, if any of the cells in this table is empty we have **missing data** and the "space" is explicitly marked. Therefore, this is a case of **structured data**. The resulting table has dimensions $M x N$, where **all rows must be of length $M$** and **all rows must be of length $N$**.

### 6.1 Creating `DataFrame` objects

We can create `DataFrame` objects from different input data types, including Python lists and dictionaries, `ndarray` objects from `NumPy`, or `Series` and `DataFrame` objects in `Pandas`. In `DataFrame` objects, both rows and columns are labeled. The labels for columns are particularly important, since they identify the variables included in our study.

In [53]:
# In this example, we illustrate different input formats for each column
# If we introduce a single value for any column, it is replicated as many
# times as needed to fill in all positions in that column.

# Note the fancy formatting for DataFrame output in Jupyter notebooks
data_1 = {
    'year': [2010, 2011, 2012, 2013, 2014] * 2 ,
    'group': ['A'] * 5 + ['B'] * 5,
    'intake': (55.3, 55.4, 55.3, 55.5, 54.4, 56.6, 57.7, 55.4, 57.9, 56),
    'output': 1.1,
    'collate': np.array([15, 5, 10, 40, 20, 12, 12, 12, 12, 12]),
    'gender': "M"
}
df_1 = pd.DataFrame(data_1)
df_1

,collate,gender,group,intake,output,year
0,15,M,A,55.3,1.1,2010
1,5,M,A,55.4,1.1,2011
2,10,M,A,55.3,1.1,2012
3,40,M,A,55.5,1.1,2013
4,20,M,A,54.4,1.1,2014
5,12,M,B,56.6,1.1,2010
6,12,M,B,57.7,1.1,2011
7,12,M,B,55.4,1.1,2012
8,12,M,B,57.9,1.1,2013
9,12,M,B,56.0,1.1,2014


Another important trait of `DataFrame` objects is that all columns must share the same indexes. In fact, `DataFrames` are created by joining `Series` objects, one for each column, and matching the corresponding index vectors.

#### Reading data from files

Other methods in `Pandas` for reading data from input files:
* `read_excel()` (files in  MS Excel format).
* `read_html()` (input data in HTML).
* `read_json()` (reading data in JSON format).
* `read_hdf()` (reading HDF5 data files).
* `read_pickle()` (reading Python objects, serialized using `cPickle`).

### 6.2 Creating `DataFrame` objects from database queries

Our example data is stored in the local MySQL database `Scratch_data` that we queried above. Remember that the `result` object stores the values returned by the query and the column names are the first elements of each object returned by `cursor.description`.

In [54]:
# The DataFrame constructor receives a list of dictionaries
# as the first argument. In each dictionary, keys identify the
# name of the column for the corresponding element.

# The columns argument receives a list of labels identifying the
# name of each columns. The columns will be created in the
# same order in which we arrange the labels of the columns argument
col_names = [x[0] for x in cursor.description]
project_df = pd.DataFrame(result, columns = col_names)

In [55]:
# Now, we can use the head() method to inspect the first 5 rows
project_df.head()

,p_ID,total-views,total-remixes
0,10519701,4.0,0.0
1,10541430,2.0,0.0
2,10942522,3.0,0.0
3,11823040,25.0,0.0
4,11828235,1.0,0.0


In [57]:
# List of columns in DataFrame object
project_df.columns

Index(['p_ID', 'total-views', 'total-remixes'], dtype='object')

In [58]:
# List of row indexes in DataFrame object
project_df.index

RangeIndex(start=0, stop=10, step=1)

In [59]:
# Creating a DataFrame from a subset of columns from
# another DataFrame. If there are non-existing columnns referenced
# in the columns argument, a new column with that label is created
# and all values in that column are filled out with 'NaN' (label for
# missing data)
df_2 = pd.DataFrame(df_1, columns=['collate', 'group', 'intake', 'genotype'])
df_2

,collate,group,intake,genotype
0,15,A,55.3,NaN
1,5,A,55.4,NaN
2,10,A,55.3,NaN
3,40,A,55.5,NaN
4,20,A,54.4,NaN
5,12,B,56.6,NaN
6,12,B,57.7,NaN
7,12,B,55.4,NaN
8,12,B,57.9,NaN
9,12,B,56.0,NaN


### 6.3 Indexing `DataFrame` objects

We can index subsets of rows or columns. For this, we can use the typical *slicing* syntax in Python with either numeric indexes or labels (if they were previously assigned).

In [79]:
# Vector of indexes
project_df.index

RangeIndex(start=0, stop=10, step=1)

In [80]:
# Values is a bidimensional ndarray (NumPy)
project_df.values

array([[  1.05197010e+07,   4.00000000e+00,   0.00000000e+00],
       [  1.05414300e+07,   2.00000000e+00,   0.00000000e+00],
       [  1.09425220e+07,   3.00000000e+00,   0.00000000e+00],
       [  1.18230400e+07,   2.50000000e+01,   0.00000000e+00],
       [  1.18282350e+07,   1.00000000e+00,   0.00000000e+00],
       [  1.20301460e+07,   1.50000000e+01,   0.00000000e+00],
       [  1.20937550e+07,   1.00000000e+00,   0.00000000e+00],
       [  1.22376150e+07,   4.00000000e+00,   0.00000000e+00],
       [  1.22564070e+07,   2.00000000e+00,   0.00000000e+00],
       [  1.24415840e+07,   3.00000000e+00,   0.00000000e+00]])

In [81]:
type(project_df.values)

numpy.ndarray

In [60]:
# Indexing by name of column
project_df['total-views']

0     4.0
1     2.0
2     3.0
3    25.0
4     1.0
5    15.0
6     1.0
7     4.0
8     2.0
9     3.0
Name: total-views, dtype: float64

In [61]:
# The result is a Series object
type(project_df['total-views'])

pandas.core.series.Series

In [62]:
# Indexing with rows and columns
project_df[:3]['total-views']

0    4.0
1    2.0
2    3.0
Name: total-views, dtype: float64

In [74]:
# This is the same as using the loc() and iloc() methods
project_df.iloc[:3].loc[: ,'total-views']

0    4.0
1    2.0
2    3.0
Name: total-views, dtype: float64

In [78]:
# Yet another way to index is using column labels or row indexes
# as attributes of the DataFrame (or Series) object

# WARNING: this will not work with labels containing blank spaces or dashes
# like some of the columns in our dataset. In those cases, consider
# renaming the columns to always use underscores '_'
project_df.p_ID

0    10519701
1    10541430
2    10942522
3    11823040
4    11828235
5    12030146
6    12093755
7    12237615
8    12256407
9    12441584
Name: p_ID, dtype: int64

In [83]:
# We can use reindexing to create a new object with their
# indexes arranged in a different way
project_df2 = project_df.reindex(columns=['total-remixes', 'total-views', 'p_ID'])
project_df2

,total-remixes,total-views,p_ID
0,0.0,4.0,10519701
1,0.0,2.0,10541430
2,0.0,3.0,10942522
3,0.0,25.0,11823040
4,0.0,1.0,11828235
5,0.0,15.0,12030146
6,0.0,1.0,12093755
7,0.0,4.0,12237615
8,0.0,2.0,12256407
9,0.0,3.0,12441584


In [84]:
# We can also drop rows or columns
# axis = 0 rows
# axis = 1 columns
project_df2 = project_df2.drop('p_ID', axis=1)
project_df2

,total-remixes,total-views
0,0.0,4.0
1,0.0,2.0
2,0.0,3.0
3,0.0,25.0
4,0.0,1.0
5,0.0,15.0
6,0.0,1.0
7,0.0,4.0
8,0.0,2.0
9,0.0,3.0


You can find additional information about this class in the [`DataFrame` documentation](http://pandas.pydata.org/pandas-docs/stable/dsintro.html#dataframe) in the `Pandas` project, as well as in the [introduction to Pandas notebook](https://github.com/fonnesbeck/Bios8366/blob/master/notebooks/Section2_1-Introduction-to-Pandas.ipynb) by C. Fonnesbeck (Vanderbilt Univ.).

### 6.4 Dealing with time series data

An important data type for `Pandas` is **time series data**. In this kind of structure, we have values associated with dates or timestamp info (sometimes only dates, other times including dates and time), indicating when each datum was collected. With this information we can build series of values along time, ordering them in a chronological fashion.

The `Pandas` library offers an [extensive collections of methods and tools to deal with time series data](http://pandas.pydata.org/pandas-docs/stable/timeseries.html), including many functions already present in other packages that have been consolidated in `Pandas`, as a jackknife for these purposes.

The first important function of this kind is `date_range`, that lets us generate ranges of labels alongg time from an initial starting point, by specifyinng the interval between equidistant points along time and the number of points to create:

In [85]:
# 48 hours since 00:00:00 of January 1, 2016
rng = pd.date_range('01/01/2016', periods=48, freq='H')
rng[:5]

DatetimeIndex(['2016-01-01 00:00:00', '2016-01-01 01:00:00',
               '2016-01-01 02:00:00', '2016-01-01 03:00:00',
               '2016-01-01 04:00:00'],
              dtype='datetime64[ns]', freq='H')

In [86]:
# Using time range to index `Series` objects, converting them into
# time series (time/value pairs)
serie_4 = pd.Series(np.random.randn(len(rng)), index=rng)
serie_4.head()

2016-01-01 00:00:00    2.620085
2016-01-01 01:00:00   -0.930885
2016-01-01 02:00:00   -1.243790
2016-01-01 03:00:00    0.029398
2016-01-01 04:00:00    0.249420
Freq: H, dtype: float64

Essentially, there are two basic forms to define a vector of values along time in `Pandas`:

* *Timestamps*: data indicating the exact date and time associated with each value point. They are generated with the  `Timestamp` class. Their associated indexes are of class `DatetimeIndex`.
* *Time spans*: they represent a single time interval, in which we indicate the starting point and the sampling frequency among the points comprising the series. The associated index is of class `PeriodIndex`.

Let's see some examples:

In [87]:
from datetime import datetime
ts_1 = pd.Timestamp(datetime(2015, 1, 1))
ts_2 = pd.Timestamp(np.datetime64('2016-06-01'))

In [88]:
ts_1

Timestamp('2015-01-01 00:00:00')

In [89]:
ts_2

Timestamp('2016-06-01 00:00:00')

In [90]:
# We can directly perform arithmetic operations on the time axis
# like substracting Timestamps, resulting in a Timedelta object
ts_2 - ts_1

Timedelta('517 days 00:00:00')

In [91]:
period_1 = pd.Period('2011-01')
period_2 = pd.Period('2011-01-01')
period_1

Period('2011-01', 'M')

In [92]:
period_2

Period('2011-01-01', 'D')

In [93]:
# Using them with `Series` object to create time series data
dates = [pd.Timestamp('2016-05-01'), pd.Timestamp('2016-05-02'), pd.Timestamp('2016-05-03')]
time_series_1 = pd.Series(np.random.randn(3), index=dates)
time_series_1

2016-05-01    0.772749
2016-05-02   -1.011647
2016-05-03    0.072712
dtype: float64

In [94]:
type(time_series_1.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [95]:
time_series_1.index

DatetimeIndex(['2016-05-01', '2016-05-02', '2016-05-03'], dtype='datetime64[ns]', freq=None)

### 6.5 Data pipelines

Some core libraries for Python scientific programming are being heavily influenced by good practices imported from other programming languages, most notably R. In the R programming language, the recent introduction of the `dplyr` and `tidyr` packages has led to a revolution in the definition and implementation of data proccessing and transformation pipelines. We can now define this sort of workflows explicitly on software code, taking advantage of other new packages like `magrittr`, which provides the `%>%` (*forward pipe*) operator. In this way, the syntax to define data processing workflows becomes even more readable and maintainable for programmers.

In Python, `Pandas` has not stayed unaware of this influence that is starting to become quite popular in data science. Operations like [group by](http://pandas.pydata.org/pandas-docs/stable/groupby.html) (following the *split-apply-combine* paradigm), [*merge* and *join*](http://pandas.pydata.org/pandas-docs/stable/merging.html), along with [transforming between *long* and *wide* data formats](http://pandas.pydata.org/pandas-docs/stable/reshaping.html) are fully supported through an ample collection of tools and methods.

Most notably, the **concatenation of operations and callables** (similar to the one available in R through the `%>%` operator) can be achieved in `Pandas` with the  [**`pipe()` method**](http://pandas.pydata.org/pandas-docs/stable/basics.html?highlight=pipe#function-application):

```python
# Let f, g, and h be methods that return a ``DataFrame`` object
>>> f(g(h(df), arg1=1), arg2=2, arg3=3)
```
```python
# Another version, this time using the pipe() method for explicit definition of data pipelines
(df.pipe(h)
       .pipe(g, arg1=1)
       .pipe(f, arg2=2, arg3=3)
    )
```

## 7 References

* [VanderPlas, 2015]. VanderPlas, J. *Python Data Science Handbook: Essential Tools for Working with Data*. O'Reilly Media, Aug. 2015.
* [McKinney, 2012] McKinney, W. *Python for Data Analysis: Data Wrangling with Pandas, NumPy, and IPython*. O'Reilly Media, Oct. 2012. **An updated edition of of this book is expected in 2018**.
* [Fonnesbeck, 2014] Fonnesbeck, C. *Advanced Statistical Computing* (Bios8366) at Vanderbilt University's Department of Biostatistics. <https://github.com/fonnesbeck/Bios8366/tree/master/>
* [Doig, 2016] Doig, C. *Scaling Data Science in Python Tutorial* (notebook 1). Available on GitHub: <https://github.com/chdoig/dss-scaling-tutorial/blob/master/1-Scaling%20Data%20Analysis/1-pandas.ipynb>.
* **Modern Pandas (Part 1 - 7)**. Blog series by [Tom Augspurger](http://tomaugspurger.github.io/pages/about.html):
    1. http://tomaugspurger.github.io/modern-1.html.
    2. http://tomaugspurger.github.io/method-chaining.html.
    3. http://tomaugspurger.github.io/modern-3-indexes.html.
    4. http://tomaugspurger.github.io/modern-4-performance.html.
    5. http://tomaugspurger.github.io/modern-5-tidy.html.
    6. http://tomaugspurger.github.io/modern-6-visualization.html.
    7. http://tomaugspurger.github.io/modern-7-timeseries.html.


- [Augspurger, 2016] Same content by T. Augspurger is available as ***"Effective Pandas"***, a [book in PDF](https://leanpub.com/effective-pandas) or a [series of notebooks on GitHub](https://github.com/TomAugspurger/effective-pandas).